# Cockpit API

In [5]:
# Import the CockpitATS client and required modules
import sys
import os
from datetime import datetime

# Add the parent directory to Python path to import clients module
parent_dir = os.path.dirname(os.getcwd())
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from clients.cockpit import CockpitATSClient

print("✅ Successfully imported CockpitATSClient")
print(f"Current time: {datetime.now()}")

cockpit_token = os.getenv("COCKPIT_KEY")
cockpit_client = CockpitATSClient("your_identity_provider_token_here")

✅ Successfully imported CockpitATSClient
Current time: 2025-11-07 14:32:57.977192


In [6]:
# Test CockpitATSClient (requires identity_provider_token)
print("Testing CockpitATSClient...")

# Example of how to test with a mock token (for demonstration purposes)
try:
    # This will fail because we need environment variables and a real token
    test_token = os.getenv("COCKPIT_KEY")
    cockpit_client = CockpitATSClient(test_token)
    
    # Test get_branch_details method
    print("\n🔍 Testing get_branch_details method...")
    try:
        branch_details = cockpit_client.get_branch_details()
        print("✅ Successfully retrieved branch details:")
        print(f"   Response type: {type(branch_details)}")
        if isinstance(branch_details, dict):
            print(f"   Number of fields: {len(branch_details)}")
            print(f"   Fields: {list(branch_details.keys())}")
        else:
            print(f"   Response: {branch_details}")
    except Exception as branch_error:
        print(f"❌ get_branch_details failed: {branch_error}")
        
except Exception as e:
    print(f"CockpitATSClient test failed: {e}")
    print("Make sure you have:")
    print("1. COCKPIT_BASE_URL environment variable set")
    print("2. A valid identity_provider_token (COCKPIT_KEY environment variable)")

Testing CockpitATSClient...

🔍 Testing get_branch_details method...
Error during Cockpit token exchange: 401 Client Error: Unauthorized for url: https://timing-prelive.recruitnowcockpit.nl/candidateprofile/api/authentication/token
❌ get_branch_details failed: 401 Client Error: Unauthorized for url: https://timing-prelive.recruitnowcockpit.nl/candidateprofile/api/authentication/token


Testing get_branch_details Method

The `get_branch_details()` method retrieves branch/owner information for the candidate from the Cockpit API endpoint: `candidatesprofile/owner`

# Planbition

In [1]:
from langchain_core.tools import tool
from pydantic import BaseModel, Field
from typing import List, Dict, Any
import sys
import os
import requests

# Add the parent directory to Python path to import clients module
parent_dir = os.path.dirname(os.getcwd())
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from clients.planbition import PlanbitionClient

# Initialize the client (this will fail if environment variables are not set)
try:
    client = PlanbitionClient()
    print("Successfully initialized PlanbitionClient")
except ValueError as e:
    print(f"Failed to initialize PlanbitionClient: {e}")
    print("Make sure the following environment variables are set:")
    print("- PLANBITION_BASE_URL")
    print("- PLANBITION_KEY") 
    print("- PLANBITION_USERNAME")
    print("- PLANBITION_PASSWORD")
    client = None

Successfully initialized PlanbitionClient


In [2]:
# Add this temporarily to your client to test
test_employee = client.get_employee_details("RE0065245")
test_employee

{'employee_id': 'RE0065245',
 'employee_number': 'RE0065245',
 'full_name': 'Arno Marsman',
 'email': 'arsinida@home.nl',
 'country': 'Nederland',
 'is_active': True}

In [3]:
test_employee = "RE0036443" 
test_start = "2025-11-07"
test_end = "2025-11-25"

schedule = client.get_employee_schedule(test_employee, test_start, test_end)
print(f"Found {len(schedule)} schedule items.")


Error calling ScheduleEmployeeShiftDemand: 500 Server Error: Internal Server Error for url: https://uat-rest.planbition.nl/api/ScheduleEmployeeShiftDemand?filter=contains%28EmployeeNumber%2C+%27RE0036443%27%29&PageNumber=1&PageSize=500
Error in get_employee_schedule: 500 Server Error: Internal Server Error for url: https://uat-rest.planbition.nl/api/ScheduleEmployeeShiftDemand?filter=contains%28EmployeeNumber%2C+%27RE0036443%27%29&PageNumber=1&PageSize=500


Found 0 schedule items.


In [4]:
schedule

[]

In [5]:
for i in range(len(schedule)):
    print("Item:", i)
    print(schedule[i]["shiftId"])
